# Capacitated Vehicle Routing Problem with Gmaps
In this session, we will solve a fictitious Capacitated VRP case using pulp library for optimization and gmaps library to create a visualization and routing. I will use the location data of the retail company for illustration of the application. Product X will be sent from Alfamidi (Depot) to six Alfamarts. It is assumed that the truck that will transport product X has a capacity of 100 units and there is a clear difference in demand between retailers.


In this capacitated VRP model there are several assumptions, including:
1. All vehicles used have the same type and capacity
2. CVRP does not consider the time window. There is a special model called VRPTW.
3. CVRP only considers the delivery capacity.

## 1. Importing Library and Collecting the Data

In [1]:
#importing library
import numpy as np
from pulp import *
import gmaps
import googlemaps
import pandas as pd

In [2]:
#Input the API key to create a map and access the distance between two places
#If you don't have the key, please register at https://developers.google.com/maps

API_KEY = 'Input your API Key here'
gmaps.configure(api_key=API_KEY)
googlemaps = googlemaps.Client(key=API_KEY)

In [3]:
#I get the data manually using gmaps
#Demand data is dummy
data = {"Name":["Depot",
                "Alfamart Bugisan",
                "Alfamart Ring Road Selatan",
                "Alfamart Bibis",
                "Alfamart UMY Tamantirto",
                "Alfamart Ngebel",
                "Alfamart Ring Road Selatan 2"],
        "Location":[(-7.820275519559818, 110.35576478748318),
                    (-7.818459939237757, 110.34828905480705),
                    (-7.826882740143658, 110.3457904293808),
                    (-7.825849387004239, 110.32770847947735),
                    (-7.814562872952285, 110.32839097560857),
                    (-7.814073787921593, 110.3183264772633),
                    (-7.809770036857429, 110.32472496772033)],
        "Demand":[0,40,50,50,30,20,10]}

#Assume the vehicle can carry 100 products
vehicle_cap = 100
alfa = pd.DataFrame(data).set_index("Name")
alfa

,Location,Demand
Name,,
Depot,"(-7.820275519559818, 110.35576478748318)",0
Alfamart Bugisan,"(-7.818459939237757, 110.34828905480705)",40
Alfamart Ring Road Selatan,"(-7.826882740143658, 110.3457904293808)",50
Alfamart Bibis,"(-7.825849387004239, 110.32770847947735)",50
Alfamart UMY Tamantirto,"(-7.814562872952285, 110.32839097560857)",30
Alfamart Ngebel,"(-7.814073787921593, 110.3183264772633)",20
Alfamart Ring Road Selatan 2,"(-7.809770036857429, 110.32472496772033)",10


## 2. Map Visualization

In [4]:
#See the location
alfa_map = gmaps.figure()

#I give a blue color for Depot
depot = gmaps.symbol_layer([alfa.Location[0]],fill_color="blue",stroke_opacity=0,
                           scale=6,info_box_content="Depot",display_info_box=True)

#Add a marker for retailer
markers = gmaps.marker_layer(alfa.Location[1:])
 
alfa_map.add_layer(depot)    
alfa_map.add_layer(markers)
    
alfa_map

Figure(layout=FigureLayout(height='420px'))

The output display will be like the image below:

<img src="https://user-images.githubusercontent.com/61647791/144628288-870c3c6a-ca83-4d02-a273-48b6cd4866be.png" />

## 3. Distance Between Two Places
The next step is to calculate the distance between two places using the API direction from gmaps. The distance that is used is the closest distance between two places with "driving" mode. Each distance between two places is then entered into a square matrix with the length and width according to the number of locations including the depot. Here is the code:

In [5]:
#Function to create distance matrix

def distance_matrix(loc_column):
    #Create a matrix with nxn size
    distance_result = np.zeros((len(loc_column),len(loc_column)))
    
    for i in range(len(loc_column)):
        for j in range(len(loc_column)):
            
            #Calculate distance between two places
            api_result = googlemaps.directions(loc_column[i],
                                               loc_column[j],
                                               mode = 'driving')
            
            #Input the calculation in the matrix 
            distance_result[i][j] = api_result[0]['legs'][0]['distance']['value']
    
    return distance_result

In [6]:
#We can convert the distance matrix into Pandas DataFrame

distance_data = pd.DataFrame(distance_matrix(alfa.Location),columns=alfa.index ,index=alfa.index)
distance_data 

Name,Depot,Alfamart Bugisan,Alfamart Ring Road Selatan,Alfamart Bibis,Alfamart UMY Tamantirto,Alfamart Ngebel,Alfamart Ring Road Selatan 2
Name,,,,,,,
Depot,0.0,1349.0,1782.0,4264.0,4693.0,5764.0,5823.0
Alfamart Bugisan,1349.0,0.0,2479.0,3536.0,3002.0,5036.0,4132.0
Alfamart Ring Road Selatan,2518.0,1217.0,0.0,2482.0,2911.0,3982.0,4041.0
Alfamart Bibis,4269.0,3552.0,3155.0,0.0,1279.0,2350.0,2409.0
Alfamart UMY Tamantirto,4667.0,3002.0,4176.0,1280.0,0.0,1689.0,1130.0
Alfamart Ngebel,6246.0,5529.0,5755.0,2827.0,1695.0,0.0,3291.0
Alfamart Ring Road Selatan 2,5633.0,4916.0,5142.0,2245.0,1070.0,1732.0,0.0


In [7]:
#You can also save the distance_data 
distance_data .to_csv("alfa_distance.csv")

## 4. Model Creation
The VRP model that I use comes from Paolo Toth and Daniele Vigo which can be seen via the paper link below:
https://www.sciencedirect.com/science/article/pii/S0166218X01003511


There are two decision variables in this model, namely Xij which is a binary type variable and indicates whether the product is sent from location i to j. The second is the variable K which indicates the number of routes used. For example, if there are three required routes, the company can allocate three vehicles to deliver products at once, or allocate one vehicle to deliver three times. The following is the mathematical model that I used:

<img src="https://user-images.githubusercontent.com/61647791/144783435-1bd7ccdf-441b-4401-a02b-857d4ccb6c5c.png" height=800 width=500 />


Paolo Toth and Daniele Vigo also provide an alternative limitation as a substitute for the **number 5** constraint which is commonly referred to as *generalized sub tour elimination constraints (GSEC)*. This constraint is much easier to translate into python code.

<img src="https://user-images.githubusercontent.com/61647791/144783942-520c9ca2-49fb-40f5-b565-a90032238bfc.png" width=500 height=300/>

In [8]:
#Initiate the model
model = LpProblem("VRP",LpMinimize)

#Decision variables
x_keys = [(i,j) for i in alfa.index for j in alfa.index]
x = LpVariable.dicts("x",x_keys,cat="Binary")
n_route = LpVariable("n_vehicle",cat="Integer") 

In [9]:
#Objective function: minimize the distance
model += lpSum(distance_data .loc[i,j]*x[i,j] for i,j in x_keys)

In [10]:
#Constraint 0
#Make sure there are no arcs from the same starting and destination locations, ex. Alfamart Bugisan to Alfamart Bugisan

for i in alfa.index:
    model += x[i,i] == 0

In [11]:
#Constraint 1: indegree constraint
#There is one path other than the Depot that goes to location j
for j in alfa.iloc[1:].index:
    model += lpSum(x[i,j] for i in alfa.index) == 1

#Constraint 2: outdegree constraint
#There is one path other than the Depot that exits location i
for i in alfa.iloc[1:].index:
    model += lpSum(x[i,j] for j in alfa.index) == 1

In [13]:
#Constraint 3: there are n paths exiting the Depot
model += lpSum(x["Depot",j] for j in alfa.index) == n_route

#Constraint 4: there are n paths that enter the Depot
model += lpSum(x[i,"Depot"] for i in alfa.index) == n_route

In [12]:
#Constraint 5: subtour elimination
#Create list untuk collect subtour
import itertools as it

subtours = []
for i in range(2,len(alfa.index)):
    subtours += it.combinations(alfa.index[1:], i)

for st in subtours:
    demand = np.sum([alfa.Demand[s] for s in st])
    model += lpSum(x[i,j] for i,j in it.permutations(st,2)) <= np.max([0,len(st)-np.ceil(demand/vehicle_cap)])

In [15]:
#fFunction to find a subtour and routing result
import copy

def get_plan(r0):
    r=copy.copy(r0)
    route = []
    while len(r) != 0:
        plan = [r[0]]
        del (r[0])
        l = 0
        while len(plan) > l:
            l = len(plan)
            for i, j in enumerate(r):
                if plan[-1][1] == j[0]:
                    plan.append(j)
                    del (r[i])
        route.append(plan)
    return(route)

In [16]:
keys = [(i,j) for i in alfa.index for j in alfa.index]
status=model.solve()
print("-----------------")
print(status,LpStatus[status],value(model.objective))
routes=[(i,j) for i,j in x_keys if value(x[i,j])==1]

-----------------
1 Optimal 21992.0


In [17]:
#Check the route
#There are two routes
get_plan(routes)

[[('Depot', 'Alfamart Bugisan'),
  ('Alfamart Bugisan', 'Alfamart UMY Tamantirto'),
  ('Alfamart UMY Tamantirto', 'Alfamart Ring Road Selatan 2'),
  ('Alfamart Ring Road Selatan 2', 'Alfamart Ngebel'),
  ('Alfamart Ngebel', 'Depot'),
  ('Depot', 'Alfamart Ring Road Selatan'),
  ('Alfamart Ring Road Selatan', 'Alfamart Bibis'),
  ('Alfamart Bibis', 'Depot')]]

In [18]:
#Separate the routes 
route_list = [[r1 for r1 in get_plan(routes)[0][:5]],[r2 for r2 in get_plan(routes)[0][5:]]]
route_list

[[('Depot', 'Alfamart Bugisan'),
  ('Alfamart Bugisan', 'Alfamart UMY Tamantirto'),
  ('Alfamart UMY Tamantirto', 'Alfamart Ring Road Selatan 2'),
  ('Alfamart Ring Road Selatan 2', 'Alfamart Ngebel'),
  ('Alfamart Ngebel', 'Depot')],
 [('Depot', 'Alfamart Ring Road Selatan'),
  ('Alfamart Ring Road Selatan', 'Alfamart Bibis'),
  ('Alfamart Bibis', 'Depot')]]

## 4. Route Visualization

In [19]:
#Visualization using Gmaps
fig = gmaps.figure()
layer = []

#Every routes have different colors
color_list = ["red","blue"]

#Draw routes
for route,color in zip(route_list,color_list):
    for r in route:
        layer.append(gmaps.directions.Directions(alfa.loc[r[0]]["Location"],
                                                 alfa.loc[r[1]]["Location"],
                                                 mode="car",stroke_weight=5, stroke_color=color,
                                                 show_markers=False))

for i in range(len(layer)):
    fig.add_layer(layer[i])    
    
    
#Add a marker
markers = gmaps.marker_layer(alfa.Location, info_box_content=alfa.index)
fig.add_layer(markers) 

fig

Figure(layout=FigureLayout(height='420px'))

The output display will be like the image below:

<img src="https://user-images.githubusercontent.com/61647791/144784928-385aa154-cda4-4241-ac92-16c921e4d20a.png" />


<img src="https://user-images.githubusercontent.com/61647791/144784983-59f705aa-5927-44c0-b179-61c1b617f57b.png" />


In [20]:
#If you want to save the map
from ipywidgets.embed import embed_minimal_html

embed_minimal_html('alfa.html', views=[fig])


## Conclusions:

1. Based on the optimization results, the total distance traveled is 21,992 km with two routes. The company can allocate two vehicles to make deliveries in one way or allocate one vehicle to make deliveries two times.
2. Route 1 is the delivery from the Depot -> Alfamart Bugisan -> Alfamart UMY Tamantirto -> Alfamart Ring Road Selatan 2 -> Alfamart Ngebel then back to the Depot. The route color is blue.
3. Route 2 is the delivery from the Depot -> South Alfamart Ring Road -> Alfamart Bibis then back to the Depot. The route color is red.
4. This CVRP model only considers the capacity of products that can be delivered by the vehicle. In addition, vehicles are assumed to be homogeneous, in the sense that they have the same type and capacity of products. There are still other VRP models as shown in the following chart:

<img src="https://www.researchgate.net/profile/Semih-Yalcindag/publication/267508322/figure/fig1/AS:295585207865357@1447484413863/Relationship-between-the-TSP-VRP-and-variants.png" />
